# Load libraries

In [3]:
#rhumba::set_channels(c("conda-forge", "default")) # Rhumba makes it easier to install packages in conda; set channels for rhumba

if (!require("psych")) rhumba::install("r-psych")
if (!require("dplyr")) rhumba::install("r-dplyr")
if (!require("ggplot2")) rhumba::install("r-ggplot2")
if (!require("car")) rhumba::install("r-car")
if (!require("janitor")) rhumba::install("r-janitor")
if (!require("readxl")) rhumba::install("r-readxl")
if (!require("remotes")) rhumba::install("r-remotes")
#if (!require("hmisc")) rhumba::install("r-hmisc")
if (!require("ggpubr")) rhumba::install("r-ggpubr")
if (!require("ggseg")) install.packages("ggseg")
if (!require("ggseg3d")) install.packages("ggseg3d")
if (!require("easystats")) install.packages("easystats")
if (!require("matrixStats")) install.packages("matrixStats")
if (!require("ggsegDKT")) install.packages("ggsegDKT")
if (!require("plotly")) install.packages("plotly")
if (!require("officer")) install.packages("officer")
if (!require("flextable")) install.packages("flextable")
if (!require("RNifti")) install.packages("RNifti")
if (!require("grid")) install.packages("grid")
if (!require("tiff")) install.packages("tiff")

library(effectsize)

Loading required package: psych

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: ggplot2


Attaching package: ‘ggplot2’


The following objects are masked from ‘package:psych’:

    %+%, alpha


Loading required package: car

Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:psych’:

    logit


Loading required package: janitor


Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test


Loading required package: readxl

Loading required package: remotes

Loading required package: ggpubr

Loading required package: ggseg

Loading required package: ggseg3d

Loading required package: easystats

# Att

In [4]:
# Custom function to round and format values
round_and_format <- function(x, digits = 4, threshold = 0.0001) {
  # Round the values first
  x_rounded <- round(x, digits)
  
  # Format values in scientific notation if they are below the threshold and non-zero
  x_formatted <- ifelse(abs(x) < threshold & abs(x) > 0, format(x, scientific = TRUE, digits = digits), x_rounded)
  
  return(x_formatted)
}


# Read in data and create variables

In [1]:
data=read.csv('/dagher/dagher11/filip/MAPT_OB//data/dataset_excluded_new.csv', header=T, quote='"')
data$genotype=NULL

In [2]:
AD_PRS=read.table('/dagher/dagher11/filip/MAPT_OB/data/UKB_AD_PRScs_zscored.csv', sep=',')
colnames(AD_PRS)=c('eid','eid2','AD_PRS')
data=merge(data, AD_PRS, by='eid', all.x=T)

## Set variables

In [5]:
columns_to_check <- c("body_mass_index_bmi_21001.2.0", "age_when_attended_assessment_centre_21003.2.0",
                     'sex_31.0.0', 'date_of_attending_assessment_centre_53.2.0','uk_biobank_assessment_centre_54.2.0',
                     'genotype_measurement_batch_22000.0.0',
                     colnames(data[grep('genetic_principal',colnames(data))][1:15]))

# Remove rows with NA values in the specified columns
data <- data[complete.cases(data[, columns_to_check]), ]

## WM structures names

In [6]:
WM_names=c("Middle Cerebellar Peduncle",
"Pontine Crossing Tract",
"Genu of Corpus Callosum",
"Body of Corpus Callosum",
"Splenium of Corpus Callosum",
"Fornix",
"Corticospinal Tract Right",
"Corticospinal Tract Left",
"Medial Lemniscus Right",
"Medial Lemniscus Left",
"Inferior Cerebellar Peduncle Right",
"Inferior Cerebellar Peduncle Left",
"Superior Cerebellar Peduncle Right",
"Superior Cerebellar Peduncle Left",
"Cerebral Peduncle Right",
"Cerebral Peduncle Left",
"Anterior Limb of Internal Capsule Right",
"Anterior Limb of Internal Capsule Left",
"Posterior Limb of Internal Capsule Right",
"Posterior Limb of Internal Capsule Left",
"Retrolenticular Part of Internal Capsule Right",
"Retrolenticular Part of Internal Capsule Left",
"Anterior Corona Radiata Right",
"Anterior Corona Radiata Left",
"Superior Corona Radiata Right",
"Superior Corona Radiata Left",
"Posterior Corona Radiata Right",
"Posterior Corona Radiata Left",
"Posterior Thalamic Radiation Right",
"Posterior Thalamic Radiation Left",
"Sagittal Stratum Right",
"Sagittal Stratum Left",
"External Capsule Right",
"External Capsule Left",
"Cingulum Cingulate Gyrus Right",
"Cingulum Cingulate Gyrus Left",
"Cingulum Hippocampus Right",
"Cingulum Hippocampus Left",
"Fornix Cres Stria Terminalis Right",
"Fornix Cres Stria Terminalis Left",
"Superior Longitudinal Fasciculus Right",
"Superior Longitudinal Fasciculus Left",
"Superior Fronto-Occipital Fasciculus Right",
"Superior Fronto-Occipital Fasciculus Left",
"Uncinate Fasciculus Right",
"Uncinate Fasciculus Left",
"Tapetum Right",
"Tapetum Left")

## Create a cardiometabolic score using PCA

In [7]:
vars_pca=data.frame(data$c.reactive_protein_30710.0.0, data$`glucose_30740.0.0`, data$`glycated_haemoglobin_hba1c_30750.0.0`, 
                    data$`systolic_blood_pressure_automated_reading_4080.0.0`,
                   data$`diastolic_blood_pressure_automated_reading_4079.0.0`, data$`triglycerides_30870.0.0`, 
                    data$`body_mass_index_bmi_21001.2.0`, data$`body_fat_percentage_23099.2.0`,
                   data$`waist_circumference_48.2.0`/data$`hip_circumference_49.2.0`, data$ldl_direct_30780.0.0, 
                    data$`cholesterol_30690.0.0`, data$`hdl_cholesterol_30760.0.0`)

In [8]:
vars_pca=scale(vars_pca)

In [9]:
a=principal(vars_pca, nfactors=1, scores = TRUE, rotate='varimax', missing=TRUE)

In [10]:
data$pca1=a$scores[,1]
#data$pca2=a$scores[,2]
#data$pca3=a$scores[,3]

# Analysis function

In [11]:
wm_model = function(genotype, term, VoI, label){
    
        
        results=matrix(ncol=9,nrow=length(c(grep('mean_fa',colnames(data)),
                                   grep('mean_md',colnames(data))))) 

        for (i in 1:length(c(grep('mean_fa',colnames(data)),
                    grep('mean_md',colnames(data))))) 
            {
            
        if (term=='main'){
            
       
    
               m=(lm(data[c(grep('mean_fa',colnames(data)),
                                       grep('mean_md',colnames(data)))][[i]] ~ 
                      unlist(data[VoI]) +
                      poly(data$age_when_attended_assessment_centre_21003.2.0, 2, raw=TRUE) *
                      data$sex_31.0.0 + 
                      poly(difftime(as.Date(data$date_of_attending_assessment_centre_53.2.0), 
                               min(as.Date(data$date_of_attending_assessment_centre_53.2.0)), units='days'), 2, raw=TRUE) +
                      data$uk_biobank_assessment_centre_54.2.0 +
                      #data$mean_rfmri_head_motion_averaged_across_space_and_time_points_25741.2.0 +
                      #data$mean_tfmri_head_motion_averaged_across_space_and_time_points_25742.2.0 +
                      data$genotype_measurement_batch_22000.0.0 +
                      ., data=subset(data, select=c(grep('principal',colnames(data)))[1:15])))

                m1=summary(m)

                results[i,1]=colnames(data)[c(grep('mean_fa',colnames(data)),
                                               grep('mean_md',colnames(data)))][i] 
                results[i,2]=m1$coefficients[2,4] # p-value 
                results[i,3]=m1$coefficients[2,3] # t-value 
                results[i,4]=m1$coefficients[2,1] # est for 
                results[i,5]=m1$coefficients[2,2] # se for 
                results[i,6]=ifelse(grepl("left", results[i, 1]), 
                        "left", 
                        ifelse(grepl("right", results[i, 1]), 
                               "right", 
                               ""))
                results[i,7]=colnames(data)[c(grep('mean_fa',colnames(data)),
                                               grep('mean_md',colnames(data)))][i] 
                results[i,8]=m1$df[2]+m1$df[1]
                results[i,9]=eta_squared(m)$Eta2_partial[2]

                }
        

    if (term=='int'){
        
        
               m=(lm(data[c(grep('mean_fa',colnames(data)),
                                       grep('mean_md',colnames(data)))][[i]] ~ 
                      unlist(data[VoI]) *
                      unlist(data[genotype])+ 
                      poly(data$age_when_attended_assessment_centre_21003.2.0, 2, raw=TRUE) *
                      data$sex_31.0.0 + 
                      poly(difftime(as.Date(data$date_of_attending_assessment_centre_53.2.0), 
                               min(as.Date(data$date_of_attending_assessment_centre_53.2.0)), units='days'), 2, raw=TRUE) +
                      data$uk_biobank_assessment_centre_54.2.0 +
                      #data$mean_rfmri_head_motion_averaged_across_space_and_time_points_25741.2.0 +
                      #data$mean_tfmri_head_motion_averaged_across_space_and_time_points_25742.2.0 +
                      data$genotype_measurement_batch_22000.0.0 +
                      ., data=subset(data, select=c(grep('principal',colnames(data)))[1:15])))

                m1=summary(m)

                results[i,1]=colnames(data)[c(grep('mean_fa',colnames(data)),
                                               grep('mean_md',colnames(data)))][i] 
                results[i,2]=m1$coefficients[131,4] # p-value 
                results[i,3]=m1$coefficients[131,3] # t-value 
                results[i,4]=m1$coefficients[131,1] # sd for 
                results[i,5]=m1$coefficients[131,2] # estimate for 
                results[i,6]=ifelse(grepl("left", results[i, 1]), 
                        "left", 
                        ifelse(grepl("right", results[i, 1]), 
                               "right", 
                               ""))
                results[i,7]=colnames(data)[c(grep('mean_fa',colnames(data)),
                                               grep('mean_md',colnames(data)))][i] 
                results[i,8]=m1$df[2]+m1$df[1]
                results[i,9]=eta_squared(m)$Eta2_partial[23]



        
    }
            }

            results[1:48,2]=p.adjust(results[1:48,2], method='BH')
            results[49:96,2]=p.adjust(results[49:96,2], method='BH')
            results=as.data.frame(results)
            results$measure=c(rep('FA', 48),rep('MD',48))
            colnames(results)=c('region','p','t','est','se','hemi','orig_label', 'Sample_size','Partial_E2','Phenotype')
            results$t=as.numeric(results$t)
            results$p=as.numeric(results$p)
            results$se=as.numeric(results$se)
            results$est=as.numeric(results$est)
            results$tract=c(rep(1:48,2))
            results$region=rep(WM_names)
            results$Partial_E2=as.numeric(results$Partial_E2)


            atlas <- readNifti('/usr/share/fsl/5.0/data/atlases/JHU/JHU-ICBM-labels-1mm.nii.gz')

            sigFA=results[(results$p<0.05 & results$Phenotype=='FA'),]
            sigMD=results[(results$p<0.05 & results$Phenotype=='MD'),]

            for (i in 1:48){
                if (i %in% sigFA$tract){atlas[atlas==i]=sigFA[sigFA$tract==i,]$t}
                else {atlas[atlas==i]=0}
                }

            writeNifti(atlas, paste("/dagher/dagher11/filip/MAPT_OB/data/sigFA_",VoI, term,".nii.gz", sep=''))

            atlas <- readNifti('/usr/share/fsl/5.0/data/atlases/JHU/JHU-ICBM-labels-1mm.nii.gz')

            for (i in 1:48){
                if (i %in% sigMD$tract){atlas[atlas==i]=sigMD[sigMD$tract==i,]$t}
                else {atlas[atlas==i]=0}
                }

            writeNifti(atlas, paste("/dagher/dagher11/filip/MAPT_OB/data/sigMD_", VoI, term,".nii.gz", sep=''))
    
            results=results %>% 
             mutate_if(is.numeric, round_and_format, digits = 4, threshold = 0.0001)
            tab=flextable(results[,c(1:3,8:10)])
            tab=autofit(tab,add_w = 0.1,add_h = 0.1,part = c("body", "header"),unit = "in")
            tab=fit_to_width(tab, max_width = 7)
            if (term=='main'){
            tab=set_caption(tab, caption = paste("Table S - WM and", VoI, 'associations', sep=' '))}
            else if (term=='int'){
            tab=set_caption(tab, caption = paste("Table S - WM*",VoI, "and", genotype, 'associations', sep=' '))}
            
            data$meanFA=rowMeans(data[results[results$p<0.05,][as.numeric(grep('mean_fa', results[results$p<0.05,]$orig_label)),]$orig_label],na.rm=T)
            data$meanMD=rowMeans(data[results[results$p<0.05,][as.numeric(grep('mean_md', results[results$p<0.05,]$orig_label)),]$orig_label],na.rm=T)
             
            data=data %>% 
              mutate(quantilegroup = ntile(body_mass_index_bmi_21001.2.0, 10)) 

            na_dat=!is.na(data[genotype])
            data$genotype_interest=data[[genotype]]
    
            FA_box = filter(data, (na_dat & !is.na(data$genotype_interest))) %>% 
                ggplot() + 
                    geom_boxplot(aes(x=as.factor(genotype_interest), y=meanFA, fill=as.factor(genotype_interest)), 
                                 show.legend=TRUE, outlier.shape=NA) +
                    theme_minimal() + 
                    ylim(0.5,0.63) +
                    ylab(paste('Mean FA in regions related to',label,sep=' ')) +
                    xlab('BMI decile') + 
                    scale_fill_manual(labels = c(paste('no',genotype,sep=' '), genotype), values = c("white", "#ADEFD1FF")) +
                    guides(fill=guide_legend(title="Genotype"))+
                        theme(axis.text.x=element_blank()) 


            MD_box = filter(data, (na_dat & !is.na(data$genotype_interest))) %>% 
                ggplot() + 
                    geom_boxplot(aes(x=as.factor(genotype_interest), y=meanMD, fill=as.factor(genotype_interest)), 
                                 show.legend=TRUE, outlier.shape=NA) +
                    theme_minimal() + 
                    ylim(0.0007,0.000855) +
                    ylab(paste('Mean MD in regions related to',label,sep=' ')) +
                    xlab('BMI decile') + 
                    scale_fill_manual(labels = c(paste('no',genotype,sep=' '), genotype), values = c("white", "#ADEFD1FF")) +
                    guides(fill=guide_legend(title="Genotype"))+
                        theme(axis.text.x=element_blank()) 


        return(list(tab, FA_box, MD_box))
         
    }


In [12]:
#data=data[!(data$body_mass_index_bmi_21001.2.0>35 | is.na(data$body_mass_index_bmi_21001.2.0)),]

https://molecularneurodegeneration.biomedcentral.com/articles/10.1186/s13024-017-0224-6

In [13]:
data$E4=dplyr::recode(data$haplotype_APOE, 'Apo-Îµ3/Îµ3'=0, 'Apo-Îµ3/Îµ4'=1, 'Apo-Îµ2/Îµ3'=0,
'Apo-Îµ1/Îµ3'=0, 'Apo-Îµ4/Îµ4'=1, 'Apo-Îµ2/Îµ2'=0)
data$H1=dplyr::recode(data$haplotype_MAPT, '0'=1, '1'=0, '2'=0)

table(data$E4)
table(data$H1)


    0     1 
25049  8666 


    0     1 
13355 20337 

# Analyses

## Analysis using function

In [14]:
genotype=c('E4','H1', 'AD_PRS')
term=c('main','int')
VoI=c('body_mass_index_bmi_21001.2.0','pca1')
label=c('Body mass index','RC 1')

for (g in 1:length(genotype)){
    for (t in 1:length(term)){
        for (v in 1:length(VoI)){
                
            assign(paste(genotype[g], term[t], VoI[v], sep='_'), 
                    wm_model(genotype[g], term[t], VoI[v], label[v]))
                
            }
        }       
    }

Warning message:
“Using one column matrices in `filter()` was deprecated in dplyr 1.1.0.
ℹ Please use one dimensional logical vectors instead.”


## Save to a table

In [15]:
save_as_docx(E4_main_body_mass_index_bmi_21001.2.0[[1]], E4_int_body_mass_index_bmi_21001.2.0[[1]], 
             H1_int_body_mass_index_bmi_21001.2.0[[1]],AD_PRS_int_body_mass_index_bmi_21001.2.0[[1]],
             E4_main_pca1[[1]], E4_int_pca1[[1]], 
             H1_int_pca1[[1]],AD_PRS_int_pca1[[1]],
             path='/dagher/dagher11/filip/MAPT_OB/Tables/WM_Tables_function_BMI_focus_AD_PRS.docx')

# Arrange all plots together

## Do this after running Plot_WM.ipynb

# Figure 2

In [19]:
genotype=c('rs7412') # Placeholder variable
term=c('main')
VoI=c('E4','H1', 'AD_PRS')
label=c('E4','H1', 'AD PRS')

for (g in 1:length(genotype)){
    for (t in 1:length(term)){
        for (v in 1:length(VoI)){
                
            assign(paste(genotype[g], term[t], VoI[v], sep='_'), 
                    wm_model(genotype[g], term[t], VoI[v], label[v]))
                
            }
        }       
    }

In [17]:
imp_figs=list()
Figs_paths=c('/dagher/dagher11/filip/MAPT_OB/figures/sigFA_E4main.tiff',
             '/dagher/dagher11/filip/MAPT_OB/figures/sigMD_E4main.tiff',
             '/dagher/dagher11/filip/MAPT_OB/figures/sigFA_H1main.tiff',
             '/dagher/dagher11/filip/MAPT_OB/figures/sigMD_H1main.tiff')
             
for (i in 1:length(Figs_paths)){
    imp_figs[[i]]=rasterGrob(readTIFF(Figs_paths[i], native=TRUE))
}

png('/dagher/dagher11/filip/MAPT_OB/figures/Figure2.png',width=4000, height=2000, res=300)
ggarrange(imp_figs[[1]], imp_figs[[2]], imp_figs[[3]], imp_figs[[4]], labels=c('a','b','c','d'), ncol=2, nrow=2)
dev.off()

png 
  2

In [20]:
save_as_docx(rs7412_main_E4[[1]], rs7412_main_H1[[1]], rs7412_main_AD_PRS[[1]], 
             path='/dagher/dagher11/filip/MAPT_OB/Tables/WM_Tables_GENETICS.docx')

## Figure 4

In [29]:
imp_figs=list()
Figs_paths=c('/dagher/dagher11/filip/MAPT_OB/figures/sigFA_body_mass_index_bmi_21001.2.0main.tiff',
             '/dagher/dagher11/filip/MAPT_OB/figures/sigMD_body_mass_index_bmi_21001.2.0main.tiff')
             
for (i in 1:length(Figs_paths)){
    imp_figs[[i]]=rasterGrob(readTIFF(Figs_paths[i], native=TRUE))
}



In [30]:
png('/dagher/dagher11/filip/MAPT_OB/figures/Figure4.png',width=4000, height=2500, res=300)
ggarrange(ggarrange(imp_figs[[1]], imp_figs[[2]],labels=c('a','b'), ncol=2, nrow=1),
          ggarrange(E4_main_body_mass_index_bmi_21001.2.0[[2]] + theme(text=element_text(size=15)), 
          E4_main_body_mass_index_bmi_21001.2.0[[3]] + theme(text=element_text(size=15)),
          H1_main_body_mass_index_bmi_21001.2.0[[2]] + theme(text=element_text(size=15)), 
          H1_main_body_mass_index_bmi_21001.2.0[[3]] + theme(text=element_text(size=15)),
          labels = c("c","d","e","f"), ncol=4),
          ncol = 1, nrow = 2, hjust = 0, heights=c(1,2))
dev.off()

png 
  2

## Figure S4

In [32]:
imp_figs=list()
Figs_paths=c('/dagher/dagher11/filip/MAPT_OB/figures/sigFA_pca1main.tiff',
             '/dagher/dagher11/filip/MAPT_OB/figures/sigMD_pca1main.tiff')
             
for (i in 1:length(Figs_paths)){
    imp_figs[[i]]=rasterGrob(readTIFF(Figs_paths[i], native=TRUE))
}

png('/dagher/dagher11/filip/MAPT_OB/figures/FigureS4.png',width=4000, height=2500, res=300)
ggarrange(imp_figs[[1]], imp_figs[[2]],labels=c('a','b'), ncol=1, nrow=2)
dev.off()

png 
  2